# Grid Search

Calling all models, far and wide!  Now is the time to battle to the DEATH!!!!

Just kidding, this is just a 

In [18]:
# notebooks use their location as their working directory, so
# if we are in a subfolder, move to the main folder.  
# This however can safely be run multiple times
#setwd(M:/lecospec/lecospec)
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)
# some features of this notebook will use this in the future, maybe.  I hope so, at least.
library(IRdisplay)

## Load the Data file names

This loads the data file names.  These are assumed to be the same across training data, labels, test data and test labels

In [23]:
BASE_PATH <- "Data/gs/"
X_TRAIN_PATH <- paste0(BASE_PATH, "x_train/")
Y_TRAIN_PATH <- paste0(BASE_PATH, "y_train/")

X_TEST_PATH <- paste0(BASE_PATH, "x_test/")
Y_TEST_PATH <- paste0(BASE_PATH, "y_test/")

In [24]:
training_data <- list.files(X_TRAIN_PATH)
#training_labels <- sort(list.files("Data/gs/y_train/"))# sort alphabetically to keep the files in the same order

In [25]:
print(training_data)

 [1] "grd_clipped_corrected.csv" "grd_clipped_minmax.csv"   
 [3] "grd_clipped_raw.csv"       "grd_clipped_robust.csv"   
 [5] "grd_clipped_standard.csv"  "grd_dropped_minmax.csv"   
 [7] "grd_dropped_raw.csv"       "grd_dropped_robust.csv"   
 [9] "grd_dropped_standard.csv"  "grd_imputed_minmax.csv"   
[11] "grd_imputed_raw.csv"       "grd_imputed_robust.csv"   
[13] "grd_imputed_standard.csv"  "grd_raw_corrected.csv"    
[15] "grd_raw_minmax.csv"        "grd_raw_raw.csv"          
[17] "grd_raw_robust.csv"        "grd_raw_standard.csv"     
[19] "img_clipped_minmax.csv"    "img_clipped_raw.csv"      
[21] "img_clipped_robust.csv"    "img_clipped_standard.csv" 
[23] "img_dropped_minmax.csv"    "img_dropped_raw.csv"      
[25] "img_dropped_robust.csv"    "img_dropped_standard.csv" 
[27] "img_imputed_minmax.csv"    "img_imputed_raw.csv"      
[29] "img_imputed_robust.csv"    "img_imputed_standard.csv" 
[31] "img_raw_minmax.csv"        "img_raw_raw.csv"          
[33] "img_raw_robust.csv

In [26]:
load_data <- function(filepath){
    data <- list()
    train_data_path <- paste0(X_TRAIN_PATH, file)
    train_labels_path <- paste0(Y_TRAIN_PATH, file)
    test_labels_path <- paste0(Y_TEST_PATH, file)
    test_data_path <- paste0(X_TEST_PATH, file)

    # verify that all the paths are valid.  If not, return NULL
    if(
        file.exists(train_data_path) &&
        file.exists(train_labels_path) &&
        file.exists(test_data_path) &&
        file.exists(test_labels_path)
    ) {
        # files all exist, so return the result
        data$x_train <- subset(read.csv(train_data_path), select=-c(X))
        data$y_train <- read.csv(train_labels_path, header = TRUE)$x %>% as.factor()
        data$x_test <- read.csv(test_data_path)
        data$y_test <- read.csv(test_labels_path, header = TRUE)$x %>% as.factor()

        return(data)
    } else {
        # a file is missing, so return NULL
        return( NULL )
    }

}

In [27]:
parse_name <- function(filepath){
    name_no_extension <- stringi::stri_replace_last_fixed(filepath, ".csv", "")
    name_parts <-strsplit(name_no_extension, "_") %>% as.vector()
    #print(name_parts)

    variables <- list(
        source = name_parts[[1]]
    )

    if(length(name_parts) > 1){
        variables$outliers <- names_parts[[2]]
    } else {
        variables$outliers <- NULL
    }

    if(length(name_parts) > 2){
        variables$preprocessing <- name_parts[[3]]
    } else {
        variables$preprocessing <- NULL
    }

    print(variables)
    return(variables)
}

In [28]:
add_model_to_manifest <- function(model_id, variables, logpath="./gs_manifest.csv"){
    if(!file.exists(logpath)){
        header <- "source,outliers,preprocessing,model_id"
        write(header, file = logpath)
    }

    line <- paste(
        variables$source %>% as.character(),
        collapse = ","
    )
    line <- paste0(line, ",", model_id)

    write(line, file=logpath, append = TRUE)
}

In [29]:
log_model_results <- function(model_id, confusion_matrix, distribition, custom, logpath = "./gs.log"){
    # append performance data to the logs for later comparison
    sink(file = logpath, append = TRUE)
    print("-------------------------------------------------------")
    print("---------------------- Model Data ---------------------")
    
    print(paste0("Model Type: Ranger (Random Forest)"))
    print(paste0("Data Index: ",custom))
    print(paste0("Model UUID: ", model_id))
    print("---------------------- Confusion Matrix ---------------------")
    print(confusion_matrix)
    print("---------------------- Class Distribution ---------------------")
    print(distribition)
    print("-------------------------------------------------------")
    sink(NULL)
}

In [31]:
for(file in training_data){

    print(file)
    # load all the data
    data <- load_data(file)
    #print(colnames(data$x_train))
    
    if(!is.null(data)){
        
        try({
        model <- ranger::ranger(
            num.trees = 1000,
            case.weights = targets_to_weights(data$y_train),
            classification = TRUE,
            x=data$x_train,
            y=data$y_train
        )

        #add forb if it's in the training data to avoid mismatch
        if(("Forb" %in% levels(data$y_train)) && !("Forb"  %in% levels(data$y_test))){
            levels(data$y_test) <- c(levels(data$y_test), "Forb")
            }

        print(levels(data$y_test))

        # create predictions (ranger)
        model_predictions <- predict(
            model, 
            data$x_test
        )$prediction %>% as.factor()

        # generate the confusion matrix
        confusion_matrix <- NULL

        confusion_matrix <- caret::confusionMatrix(
            model_predictions, 
            data$y_test,
            mode = "everything"
        )

        # generate an id to uniquely identify the model
        model_id <- uuid::UUIDgenerate()

        # append performance data to the logs for later comparison
        log_model_results(
            model_id = model_id,
            confusion_matrix = confusion_matrix,
            custom = file,
            distribition = model_predictions %>% as.factor() %>% table())

        # track what levels are associated with the UUID
        add_model_to_manifest(
            model_id = model_id,
            variables = parse_name(file)
        )

        # save the model using the model UUID
        save(model, file = paste0("mle/models/gs/", model_id, ".rda"))
        })
    }
}

[1] "grd_clipped_corrected.csv"
[1] "grd_clipped_minmax.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
[9] "Forb"          


Warning message in confusionMatrix.default(model_predictions, data$y_test, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”


$source
[1] "grd"     "clipped" "minmax" 

[1] "grd_clipped_raw.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
[9] "Forb"          


Warning message in confusionMatrix.default(model_predictions, data$y_test, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”


$source
[1] "grd"     "clipped" "raw"    

[1] "grd_clipped_robust.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
[9] "Forb"          


...

Error in table(data, reference, dnn = dnn, ...) : 
  all arguments must have the same length
[1] "img_dropped_robust.csv"
[1] "Abiotic"        "Forb"           "Graminoid"      "Lichen"        
[5] "Moss"           "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf" 
[9] "TreeConifer"   


Warning message in levels(reference) != levels(data):
“longer object length is not a multiple of shorter object length”
Warning message in confusionMatrix.default(model_predictions, data$y_test, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”


Error in table(data, reference, dnn = dnn, ...) : 
  all arguments must have the same length
[1] "img_dropped_standard.csv"
[1] "Abiotic"        "Forb"           "Graminoid"      "Lichen"        
[5] "Moss"           "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf" 
[9] "TreeConifer"   


Warning message in levels(reference) != levels(data):
“longer object length is not a multiple of shorter object length”
Warning message in confusionMatrix.default(model_predictions, data$y_test, mode = "everything"):
“Levels are not in the same order for reference and data. Refactoring data to match.”


Error in table(data, reference, dnn = dnn, ...) : 
  all arguments must have the same length
[1] "img_imputed_minmax.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
$source
[1] "img"     "imputed" "minmax" 

[1] "img_imputed_raw.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
$source
[1] "img"     "imputed" "raw"    

[1] "img_imputed_robust.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
$source
[1] "img"     "imputed" "robust" 

[1] "img_imputed_standard.csv"
[1] "Abiotic"        "Graminoid"      "Lichen"         "Moss"          
[5] "ShrubDecid"     "ShrubEvergreen" "TreeBroadleaf"  "TreeConifer"   
$source
[1] "img"      "imputed"  "standard"

[1] "img_raw_minmax.csv"
[1] "Abiotic"   

In [12]:
print((confusion_matrix$overall %>% as.list())$Accuracy)

[1] 1


In [13]:
length(data$y_test)
nrow(data$x_test)

[1] 120

[1] 120

In [ ]:
length(data$y_train)
nrow(data$x_train)

In [15]:
clip_transform <- load_model(
    "./mle/clip_transform.rda"
)

In [16]:
clip_transform

function (df, ignore_cols = NULL) 
{
    new_df <- df %>% as.data.frame()
    cache_path <- "./assets/clip_data.json"
    fences <- rjson::fromJSON(file = cache_path)
    used_cols <- colnames(df)
    if (!is.null(ignore_cols)) {
        used_cols <- setdiff(intersect(names(fences), colnames(df)), 
            ignore_cols)
    }
    for (column_name in used_cols) {
        upper_fence <- fences[column_name]$upper
        lower_fence <- fences[column_name]$lower
        high_outliers <- df[, column_name] > upper_fence
        low_outliers <- df[, column_name] < lower_fence
        new_df[high_outliers, column_name] = upper_fence
        new_df[low_outliers, column_name] = lower_fence
    }
    return(new_df)
}
<bytecode: 0x5557f7757058>
<environment: 0x555801c17818>

In [32]:
model_index <- read.csv("./gs_manifest.csv")
model_index

source,outliers,preprocessing,model_id
<chr>,<chr>,<chr>,<chr>
grd,clipped,minmax,b5c8d838-c36c-4173-bb5b-7ddc91b685da
grd,clipped,raw,4750c1b3-54bd-44cf-be87-27694c109043
grd,clipped,robust,abc6db2a-23a5-40f8-bae7-2b82bccdc0dd
grd,clipped,standard,d68c5641-4197-4ec0-8103-34a256fe3203
grd,clipped,minmax,8a5a9be2-23f8-4752-ba42-a0592c4ecdae
grd,clipped,raw,cd46c134-bfc9-4938-a995-6e3fda63fba6
grd,clipped,robust,6251e8eb-2176-4ea6-bff0-5d91fb307ee3
grd,clipped,standard,41e7a280-412f-4aa7-a97b-15846d8b6b17
grd,imputed,minmax,b24c62a5-5bc9-4b32-98ee-7052773737e2


In [32]:
print(model_index[17,])

   source outliers preprocessing                             model_id
17    img  clipped        minmax 0831a1b8-80f1-43b0-98f6-878e5dbfbf57


TO DO: Add preprocessing selection to the next cell!

In [38]:
#for(i in seq(nrow(model_index))){
    #i <- 17
    #sink(NULL)
    #sink(file = "./gs_validation.log", append = TRUE)
    
    id <-"f6cdee94-d3f5-43b5-8315-a26553785d0e" #model_index[i,"model_id"]
    model <- load_model(
        paste0("mle/models/gs/", id, ".rda")
    )

    print(id)


    save_path <- paste0("mle/experiments/gs/", id, "/")
    #print(names(model))
    #print(model)
    if(!dir.exists(save_path)){
        dir.create(save_path)
    } 
    #print(quadrats[[3]])
    # keep the notebook tidy by writing the out
    #print(id)
    results <- validate_model(
        model, 
        save_path, 
        outlier_processing = "none",
        transform_type = "none",
        cluster = NULL)
    aggregated_results <- aggregate_results(save_path)

    plot_by_pft(
        aggregated_results,
        save_path = paste0(save_path, "aggregate.html"),
        open = FALSE
    )
#
    write_validation_table(
        aggregated_results,
        save_path = paste0(save_path, "table.html"),
        open = FALSE
    )

    #sink(NULL)
#}

[1] "f6cdee94-d3f5-43b5-8315-a26553785d0e"
[1] "preprocessing raster at Data/Ground_Validation/Imagery/BisonGulchQuads.envi"
[1] "Converted to Data frame?"
[1] TRUE
[1] "Noisy columns removed"
[1] TRUE
[1] "Filtered"
[1] TRUE


Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



[1] "Imputing..."
[1] "Handling Outliers with method: none"
[1] "Transforming the data with transform: none"
          x        y z
1 -148.9508 63.80701 6
2 -148.9508 63.80701 2
3 -148.9508 63.80701 5
4 -148.9508 63.80701 2
5 -148.9508 63.80701 2
6 -148.9508 63.80701 5
          x        y z
1 -148.9508 63.80701 6
2 -148.9508 63.80701 2
3 -148.9508 63.80701 5
4 -148.9508 63.80701 2
5 -148.9508 63.80701 2
6 -148.9508 63.80701 5
[1] "Attempting to save to ./validation_saved_output.grd"
[1] "Converted to Raster"
Reading layer `Bisoon_Quadrats_georeferenced' from data source 
  `/home/krbundy/GitHub/lecospec/Data/Vectors/Bisoon_Quadrats_georeferenced.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 9 features and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -148.9524 ymin: 63.80698 xmax: -148.9508 ymax: 63.80701
Geodetic CRS:  WGS 84
Simple feature collection with 1 feature and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xm

Saving 6.67 x 6.67 in image



Simple feature collection with 1 feature and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -148.952 ymin: 63.80699 xmax: -148.952 ymax: 63.807
Geodetic CRS:  GCS_unknown
  CLASS_ID   CLASS_NAME AREA                       geometry
2        7 Bisongulch70  649 POLYGON ((-148.952 63.80699...


...

Saving 6.67 x 6.67 in image



Simple feature collection with 1 feature and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -148.2972 ymin: 64.70356 xmax: -148.2972 ymax: 64.70357
Geodetic CRS:  GCS_unknown
   CLASS_NAME CLASS_ID CLASS_CLRS                       geometry
10   Bonanza0        1    255,0,0 POLYGON ((-148.2972 64.7035...


Saving 6.67 x 6.67 in image



Simple feature collection with 1 feature and 3 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -148.2965 ymin: 64.70411 xmax: -148.2965 ymax: 64.70412
Geodetic CRS:  GCS_unknown
   CLASS_NAME CLASS_ID CLASS_CLRS                       geometry
11  Bonanza70        8    0,255,0 POLYGON ((-148.2965 64.7041...


Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

